### Fee Calculator API Fee Retrieve
Retrieves some standard fee results from LAA Fee Calculator    
Store results in DataFrame and display with some highlighting     

**Results are displayed near the end**

*Launching local fee calc*    
`DEBUG="True" venv/bin/python manage.py runserver 8000`

**Prices**
https://laa-fee-calculator.service.justice.gov.uk/api/v1/fee-schemes/4/prices

**Docs**
https://laa-fee-calculator.service.justice.gov.uk/api/v1/docs/

**GitHub**
https://github.com/ministryofjustice/laa-fee-calculator

### Setup some stuff

In [21]:
import time
import requests
import pandas as pd
from IPython.display import Markdown

In [22]:
def get_fee(scheme = 4,
            fee_type="LIT_FEE",
            scenario="1",
            offence_class="A",
            days="1",
            ppe="0",
            app_url = "http://localhost:8000/"):
    """Get free using LAA Fee Calculator API"""
    
    # create end-part of URL
    end = ("api/v1/fee-schemes/{}/calculate/"
           "?fee_type_code={}&scenario={}&offence_class={}&days={}&ppe={}"
           ).format(scheme, fee_type, scenario, offence_class, days, ppe)
 
    response = requests.get(app_url + end)
    if response.status_code == 200:
        # .json method returns a dictionary - use to extract just the amount
        return response.json().get("amount")
    else:
        return "error: {} - {}".format(response.status_code, response.text)

In [24]:
def get_info_as_dataframe(url_end="fee-schemes/4/fee-types/",
                          url_start="https://laa-fee-calculator.service.justice.gov.uk/api/v1/"):
    """Fairly generic way of getting response from fee calc and storing as a dataframe"""
    url = url_start + url_end
    response = requests.get(url)
    df = pd.DataFrame(response.json().get("results"))
    return df

In [23]:
def highlight_conditions(val):
    """Pandas dataframe conditional highlighting function
    Used with df.style.applymap method.
    """
    colour = "black"
    if "error" in str(val):
        colour = "red"
    return 'color: %s' % colour

def display_full_df(df, heading=""):
    """Display whole of data frame content - avoids auto truncation"""
    if heading:
        display(Markdown("#### " + str(heading)))
    with pd.option_context('display.max_columns', None, 'display.max_rows', None, 'display.max_colwidth', -1):
        display(df)

### Fee Schemes

In [4]:
# Get info about the schemes
response = requests.get("https://laa-fee-calculator.service.justice.gov.uk/api/v1/fee-schemes/")

In [32]:
columns = ["id", "description", "type", "start_date", "end_date"]
for result in response.json().get("results"):
    print(result["id"], result["description"],result["type"], result["start_date"], result["end_date"])

(1, u'AGFS Fee Scheme 9', u'AGFS', u'2012-04-01', u'2018-03-31')
(2, u'LGFS Fee Scheme 2016-04', u'LGFS', u'2016-04-01', None)
(3, u'AGFS Fee Scheme 10', u'AGFS', u'2018-04-01', u'2018-12-30')
(4, u'AGFS Fee Scheme 11', u'AGFS', u'2018-12-31', None)


In [5]:
# More convenient? Feed json to dataframe
scheme_df = pd.DataFrame(response.json().get("results"))
#Change column order,yes [[]]
scheme_df = scheme_df[["id", "description", "type", "start_date", "end_date"]]
display(scheme_df)

,id,description,type,start_date,end_date
0,1,AGFS Fee Scheme 9,AGFS,2012-04-01,2018-03-31
1,2,LGFS Fee Scheme 2016-04,LGFS,2016-04-01,None
2,3,AGFS Fee Scheme 10,AGFS,2018-04-01,2018-12-30
3,4,AGFS Fee Scheme 11,AGFS,2018-12-31,None


#### Fairly generic way of getting stuff?

In [25]:
things = ["fee-schemes/4/modifier-types/", "fee-schemes/4/fee-types/", "fee-schemes/4/prices/"]
for thing in things:
    df = get_info_as_dataframe(thing)
    display_full_df(df, thing)

#### fee-schemes/4/modifier-types/

,description,id,name,unit
0,Number of cases,1,NUMBER_OF_CASES,CASE
1,Number of defendants,2,NUMBER_OF_DEFENDANTS,DEFENDANT
2,Trial length,3,TRIAL_LENGTH,DAY
3,Pages of prosecuting evidence,4,PAGES_OF_PROSECUTING_EVIDENCE,PPE
4,Whole months between trials,5,RETRIAL_INTERVAL,MONTH
5,Third in which a trial cracked,6,THIRD_CRACKED,THIRD
6,Whole months between warrant issue and execution,7,WARRANT_INTERVAL,MONTH


#### fee-schemes/4/fee-types/

,aggregation,code,id,is_basic,name
0,sum,AGFS_ABS_PRC_HF,5,False,Abuse of process hearing (half day)
1,sum,AGFS_ABS_PRC_WL,6,False,Abuse of process hearing (full day)
2,sum,AGFS_ADJOURNED,7,False,Adjourned appeals
3,sum,AGFS_APPEAL_CON,8,False,Appeals to crown court against conviction
4,sum,AGFS_APPEAL_SEN,9,False,Appeals to the crown court against sentence
5,sum,AGFS_DMS_DY2_HF,10,False,Application to dismiss a charge day 2 onwards (half day)
6,sum,AGFS_DMS_DY2_WL,11,False,Application to dismiss a charge day 2 onwards (full day)
7,sum,AGFS_COMMITTAL,12,False,Committal for sentence hearing
8,sum,AGFS_CONFERENCE,13,False,Conferences and views (hours)
9,sum,AGFS_CONTEMPT,14,False,Contempt hearings


#### fee-schemes/4/prices/

,advocate_type,fee_per_unit,fee_type,fixed_fee,id,limit_from,limit_to,modifiers,offence_class,scenario,scheme,strict_range,unit
0,QC,0.00000,34,1010.00000,110393,1,NaN,"[{u'fixed_percent': u'0.00', u'required': True, u'priority': 0, u'strict_range': False, u'limit_to': None, u'modifier_type': {u'description': u'Whole months between warrant issue and execution', u'id': 7, u'unit': u'MONTH', u'name': u'WARRANT_INTERVAL'}, u'percent_per_unit': u'0.00', u'limit_from': 3}]",6.4,47,4,False,DAY
1,QC,0.00000,34,810.00000,110394,1,NaN,"[{u'fixed_percent': u'0.00', u'required': True, u'priority': 0, u'strict_range': False, u'limit_to': None, u'modifier_type': {u'description': u'Whole months between warrant issue and execution', u'id': 7, u'unit': u'MONTH', u'name': u'WARRANT_INTERVAL'}, u'percent_per_unit': u'0.00', u'limit_from': 3}]",6.5,47,4,False,DAY
2,QC,0.00000,34,1415.00000,110395,1,NaN,"[{u'fixed_percent': u'0.00', u'required': True, u'priority': 0, u'strict_range': False, u'limit_to': None, u'modifier_type': {u'description': u'Whole months between warrant issue and execution', u'id': 7, u'unit': u'MONTH', u'name': u'WARRANT_INTERVAL'}, u'percent_per_unit': u'0.00', u'limit_from': 3}]",7.1,47,4,False,DAY
3,QC,0.00000,34,810.00000,110396,1,NaN,"[{u'fixed_percent': u'0.00', u'required': True, u'priority': 0, u'strict_range': False, u'limit_to': None, u'modifier_type': {u'description': u'Whole months between warrant issue and execution', u'id': 7, u'unit': u'MONTH', u'name': u'WARRANT_INTERVAL'}, u'percent_per_unit': u'0.00', u'limit_from': 3}]",7.2,47,4,False,DAY
4,QC,0.00000,34,760.00000,110397,1,NaN,"[{u'fixed_percent': u'0.00', u'required': True, u'priority': 0, u'strict_range': False, u'limit_to': None, u'modifier_type': {u'description': u'Whole months between warrant issue and execution', u'id': 7, u'unit': u'MONTH', u'name': u'WARRANT_INTERVAL'}, u'percent_per_unit': u'0.00', u'limit_from': 3}]",7.3,47,4,False,DAY
5,QC,0.00000,34,1210.00000,110398,1,NaN,"[{u'fixed_percent': u'0.00', u'required': True, u'priority': 0, u'strict_range': False, u'limit_to': None, u'modifier_type': {u'description': u'Whole months between warrant issue and execution', u'id': 7, u'unit': u'MONTH', u'name': u'WARRANT_INTERVAL'}, u'percent_per_unit': u'0.00', u'limit_from': 3}]",8.1,47,4,False,DAY
6,QC,0.00000,34,5860.00000,110399,1,NaN,"[{u'fixed_percent': u'0.00', u'required': True, u'priority': 0, u'strict_range': False, u'limit_to': None, u'modifier_type': {u'description': u'Whole months between warrant issue and execution', u'id': 7, u'unit': u'MONTH', u'name': u'WARRANT_INTERVAL'}, u'percent_per_unit': u'0.00', u'limit_from': 3}]",9.1,47,4,False,DAY
7,QC,0.00000,34,4040.00000,110400,1,NaN,"[{u'fixed_percent': u'0.00', u'required': True, u'priority': 0, u'strict_range': False, u'limit_to': None, u'modifier_type': {u'description': u'Whole months between warrant issue and execution', u'id': 7, u'unit': u'MONTH', u'name': u'WARRANT_INTERVAL'}, u'percent_per_unit': u'0.00', u'limit_from': 3}]",9.2,47,4,False,DAY
8,QC,0.00000,34,3030.00000,110401,1,NaN,"[{u'fixed_percent': u'0.00', u'required': True, u'priority': 0, u'strict_range': False, u'limit_to': None, u'modifier_type': {u'description': u'Whole months between warrant issue and execution', u'id': 7, u'unit': u'MONTH', u'name': u'WARRANT_INTERVAL'}, u'percent_per_unit': u'0.00', u'limit_from': 3}]",9.3,47,4,False,DAY
9,QC,0.00000,34,2650.00000,110402,1,NaN,"[{u'fixed_percent': u'0.00', u'required': True, u'priority': 0, u'strict_range': False, u'limit_to': None, u'modifier_type': {u'description': u'Whole months between warrant issue and execution', u'id': 7, u'unit': u'MONTH', u'name': u'WARRANT_INTERVAL'}, u'percent_per_unit': u'0.00', u'limit_from': 3}]",9.4,47,4,False,DAY


### Interact with API to extract a range of responses and store in DataFrame

Lit Fee, Scenario 1, 1-day duration    
Varying Offence Codes and PPE values (including some invalid values).

In [6]:
#DataFrame to hold results
df = pd.DataFrame(columns=["Offence Code", "PPE", "Result"])

app_url = "http://localhost:8000/"
app_url = 'https://laa-fee-calculator.service.justice.gov.uk/'

start_time = time.strftime("%d/%m/%Y %H:%M:%S")
for oc in "ABCDEFGHIJKL":
    for ppe in [-81, -80, -1, 0, 1, 80, 81, "z"]:
        result = get_fee(app_url=app_url, offence_class=oc, ppe=ppe)
        row_data = {"Offence Code":oc, "PPE":ppe, "Result":result}
        df = df.append(row_data, ignore_index=True)

### Display Results

In [55]:
# Highlight errors in red
highlighted = df.style.applymap(highlight_conditions)
# Turn of row display limit
pd.options.display.max_rows = None
# Display results
display(start_time)
display(highlighted)

'22/11/2018 10:19:02'

,Offence Code,PPE,Result
0,A,-81,340.2
1,A,-80,340.2
2,A,-1,340.2
3,A,0,340.2
4,A,1,340.2
5,A,80,340.2
6,A,81,343
7,A,z,"error: 400 - [""`ppe` must be a number""]"
8,B,-81,278.06
9,B,-80,278.06
